## __Question 01__

Datasets binomial-classification-v1.csv and binomial-classification-v2.csv contain actuals and predictions values from a binomial response variable.

1. Print one record containing a true/positive, true/negative, false/positive, false/negative <br>
2. Manually compute accuracy, precision, recall, f1_Score <br>
3. Compute the metrics in item 2 using sklearn and tensorflow, and torch <br>
4. Show the values matching across all three computing approaches <br>

### __For binomial-classification-v1.csv (V1)__

In [24]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, mean_absolute_percentage_error
import tensorflow as tf
import torch

In [5]:
v1_df = pd.read_csv('/Users/veeraanand/Documents/Career/UChicago MS-ADS/2024-2025_MS Program/2025 Spring/Machine Learning II/Wk 1/binomial-classification-v1.csv')
v1_df = v1_df[['actual', 'predicted']]

v2_df = pd.read_csv('/Users/veeraanand/Documents/Career/UChicago MS-ADS/2024-2025_MS Program/2025 Spring/Machine Learning II/Wk 1/binomial-classification-v2.csv')
v2_df = v2_df[['actual', 'predicted']]

print(v1_df)
print(v2_df)

      actual  predicted
0          1          0
1          0          1
2          1          0
3          0          1
4          1          0
...      ...        ...
9994       1          0
9995       0          1
9996       1          0
9997       0          1
9998       1          0

[9999 rows x 2 columns]
      actual  predicted
0          0          0
1          0          0
2          0          0
3          0          0
4          0          0
...      ...        ...
9994       1          1
9995       1          1
9996       1          1
9997       1          1
9998       1          1

[9999 rows x 2 columns]


In [10]:
def compute_metrics(actual, predicted):
    TP = np.where((actual == 1) & (predicted == 1),1,0)
    TN = np.where((actual == 0) & (predicted == 0),1,0)
    FP = np.where((actual == 0) & (predicted == 1),1,0)
    FN = np.where((actual == 1) & (predicted == 0),1,0)
    metrics = pd.DataFrame()
    metrics["actual"] = actual
    metrics["predicted"] = predicted
    metrics["TP"] = TP
    metrics["TN"] = TN
    metrics["FP"] = FP
    metrics["FN"] = FN
    return metrics, TP, TN, FP, FN

v1_metrics, TP, TN, FP, FN = compute_metrics(v1_df['actual'], v1_df['predicted'])
display(v1_metrics)

,actual,predicted,TP,TN,FP,FN
0,1,0,0,0,0,1
1,0,1,0,0,1,0
2,1,0,0,0,0,1
3,0,1,0,0,1,0
4,1,0,0,0,0,1
...,...,...,...,...,...,...
9994,1,0,0,0,0,1
9995,0,1,0,0,1,0
9996,1,0,0,0,0,1
9997,0,1,0,0,1,0


In [7]:
tn = v1_metrics.loc[v1_metrics['TN'] == 1][:1]
display(tn)
tp = v1_metrics.loc[v1_metrics['TP'] == 1][:1]
display(tp)
fn = v1_metrics.loc[v1_metrics['FN'] == 1][:1]
display(fn)
fp = v1_metrics.loc[v1_metrics['FP'] == 1][:1]
display(fp)

,actual,predicted,TP,TN,FP,FN
1,0,0,0,1,0,0


,actual,predicted,TP,TN,FP,FN
11,1,1,1,0,0,0


,actual,predicted,TP,TN,FP,FN
0,1,0,0,0,0,1


,actual,predicted,TP,TN,FP,FN
12,0,1,0,0,1,0


In [25]:
TP = np.sum(TP)
TN = np.sum(TN)
FP = np.sum(FP)
FN = np.sum(FN)

# manual
accuracy_mn = (TP+TN)/(TP+TN+FP+FN)
precision_mn = (TP)/(TP+FP)
recall_mn = TP/(TP+FN)
f1_score_mn = 2*(precision_mn*recall_mn)/(precision_mn+recall_mn)
specificity_mn = TN/(TN+FP)

# sklearn
accuracy_skl = accuracy_score(v1_df['actual'], v1_df['predicted'])
precision_skl = precision_score(v1_df['actual'], v1_df['predicted'])
recall_skl = recall_score(v1_df['actual'], v1_df['predicted'])
f1_score_skl = f1_score(v1_df['actual'], v1_df['predicted'])

# tensorflow
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(v1_df['actual'], v1_df['predicted'])
accuracy_tf = accuracy.result().numpy()

precision = tf.keras.metrics.Precision()
precision.update_state(v1_df['actual'], v1_df['predicted'])
precision_tf = precision.result().numpy()

recall = tf.keras.metrics.Recall()
recall.update_state(v1_df['actual'], v1_df['predicted'])
recall_tf = recall.result().numpy()

f1_score_TF = 2 * (precision_tf * recall_tf) / (precision_tf + recall_tf)


# torch
y_actual_tensor = torch.tensor(v1_df['actual'].values)
y_predic_tensor = torch.tensor(v1_df['predicted'].values)

TP_torch = ((y_actual_tensor == 1) & (y_predic_tensor == 1)).sum().item()
TN_torch = ((y_actual_tensor == 0) & (y_predic_tensor == 0)).sum().item()
FP_torch = ((y_actual_tensor == 0) & (y_predic_tensor == 1)).sum().item()
FN_torch = ((y_actual_tensor == 1) & (y_predic_tensor == 0)).sum().item()

accuracy_torch = (TP_torch + TN_torch) / (TP_torch + TN_torch + FP_torch + FN_torch)
precision_torch = TP_torch / (TP_torch + FP_torch) if (TP_torch + FP_torch) > 0 else 0
recall_torch = TP_torch / (TP_torch + FN_torch) if (TP_torch + FN_torch) > 0 else 0
f1_score_torch = 2 * (precision_torch * recall_torch) / (precision_torch + recall_torch) if (precision_torch + recall_torch) > 0 else 0


compareResults = \
    pd.DataFrame(
    {
     "metric": ["accuracy", "precision", "recall", "f1_score"],
     "manual": [accuracy_mn, precision_mn, recall_mn, f1_score_mn],
     "sklearn": [accuracy_skl, precision_skl, recall_skl, f1_score_skl],
     "tensorflow": [accuracy_tf, precision_tf, recall_tf, f1_score_TF],
     "torch": [accuracy_torch, precision_torch, recall_torch, f1_score_torch]
    }
    )

compareResults

,metric,manual,sklearn,tensorflow,torch
0,accuracy,0.948495,0.048605,0.048605,0.048605
1,precision,0.948876,0.046512,0.046512,0.046512
2,recall,0.948687,0.046502,0.046502,0.046502
3,f1_score,0.948782,0.046507,0.046507,0.046507


### __For binomial-classification-v2.csv (V2)__

In [11]:
v2_df = pd.read_csv('/Users/veeraanand/Documents/Career/UChicago MS-ADS/2024-2025_MS Program/2025 Spring/Machine Learning II/Wk 1/binomial-classification-v2.csv')
v2_df = v2_df[['actual', 'predicted']]
print(v2_df)

v2_metrics, TP, TN, FP, FN = compute_metrics(v2_df['actual'], v2_df['predicted'])
display(v2_metrics)

      actual  predicted
0          0          0
1          0          0
2          0          0
3          0          0
4          0          0
...      ...        ...
9994       1          1
9995       1          1
9996       1          1
9997       1          1
9998       1          1

[9999 rows x 2 columns]


,actual,predicted,TP,TN,FP,FN
0,0,0,0,1,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,0,0,1,0,0
4,0,0,0,1,0,0
...,...,...,...,...,...,...
9994,1,1,1,0,0,0
9995,1,1,1,0,0,0
9996,1,1,1,0,0,0
9997,1,1,1,0,0,0


In [12]:
tn = v2_metrics.loc[v2_metrics['TN'] == 1][:1]
display(tn)
tp = v2_metrics.loc[v2_metrics['TP'] == 1][:1]
display(tp)
fn = v2_metrics.loc[v2_metrics['FN'] == 1][:1]
display(fn)
fp = v2_metrics.loc[v2_metrics['FP'] == 1][:1]
display(fp)

,actual,predicted,TP,TN,FP,FN
0,0,0,0,1,0,0


,actual,predicted,TP,TN,FP,FN
11,1,1,1,0,0,0


,actual,predicted,TP,TN,FP,FN
10,1,0,0,0,0,1


,actual,predicted,TP,TN,FP,FN
79,0,1,0,0,1,0


In [26]:
TP = np.sum(TP)
TN = np.sum(TN)
FP = np.sum(FP)
FN = np.sum(FN)

# manual
accuracy_mn = (TP+TN)/(TP+TN+FP+FN)
precision_mn = (TP)/(TP+FP)
recall_mn = TP/(TP+FN)
f1_score_mn = 2*(precision_mn*recall_mn)/(precision_mn+recall_mn)
specificity_mn = TN/(TN+FP)

# sklearn
accuracy_skl = accuracy_score(v2_df['actual'], v2_df['predicted'])
precision_skl = precision_score(v2_df['actual'], v2_df['predicted'])
recall_skl = recall_score(v2_df['actual'], v2_df['predicted'])
f1_score_skl = f1_score(v2_df['actual'], v2_df['predicted'])

# tensorflow
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(v2_df['actual'], v2_df['predicted'])
accuracy_tf = accuracy.result().numpy()

precision = tf.keras.metrics.Precision()
precision.update_state(v2_df['actual'], v2_df['predicted'])
precision_tf = precision.result().numpy()

recall = tf.keras.metrics.Recall()
recall.update_state(v2_df['actual'], v2_df['predicted'])
recall_tf = recall.result().numpy()

f1_score_TF = 2 * (precision_tf * recall_tf) / (precision_tf + recall_tf)

#torch
y_true_tensor = torch.tensor(v2_df['actual'].values)
y_pred_tensor = torch.tensor(v2_df['predicted'].values)

TP_torch = ((y_true_tensor == 1) & (y_pred_tensor == 1)).sum().item()
TN_torch = ((y_true_tensor == 0) & (y_pred_tensor == 0)).sum().item()
FP_torch = ((y_true_tensor == 0) & (y_pred_tensor == 1)).sum().item()
FN_torch = ((y_true_tensor == 1) & (y_pred_tensor == 0)).sum().item()

accuracy_torch = (TP_torch + TN_torch) / (TP_torch + TN_torch + FP_torch + FN_torch)
precision_torch = TP_torch / (TP_torch + FP_torch) if (TP_torch + FP_torch) > 0 else 0
recall_torch = TP_torch / (TP_torch + FN_torch) if (TP_torch + FN_torch) > 0 else 0
f1_score_torch = 2 * (precision_torch * recall_torch) / (precision_torch + recall_torch) if (precision_torch + recall_torch) > 0 else 0

# Final comparison
compareResults = pd.DataFrame({
    "metric": ["accuracy", "precision", "recall", "f1_score"],
    "manual": [accuracy_mn, precision_mn, recall_mn, f1_score_mn],
    "sklearn": [accuracy_skl, precision_skl, recall_skl, f1_score_skl],
    "tensorflow": [accuracy_tf, precision_tf, recall_tf, f1_score_TF],
    "torch": [accuracy_torch, precision_torch, recall_torch, f1_score_torch]
})

compareResults

,metric,manual,sklearn,tensorflow,torch
0,accuracy,0.948495,0.948495,0.948495,0.948495
1,precision,0.948876,0.948876,0.948876,0.948876
2,recall,0.948687,0.948687,0.948687,0.948687
3,f1_score,0.948782,0.948782,0.948782,0.948782
